This is a cover letter builder using agentic ai


In [111]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import json

In [112]:
load_dotenv(override=True)


True

In [113]:
openai = OpenAI()

In [114]:
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [115]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [116]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [117]:
name = "Sviatoslav Rutkovskyi"


In [118]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover lettter. 
Be professional and engaging, using the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
"""



In [119]:

updated_system_prompt = system_prompt

In [120]:
evaluator_system_prompt = f"""
You are an evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter does not include any markdown and formatting besides the stardart 3 paragraph format. 
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [121]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [122]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your extensive feedback.
    """

In [123]:
def update_system_prompt(cover_letter, feedback):
        global updated_system_prompt
        updated_system_prompt = updated_system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """
        return updated_system_prompt;

In [124]:
def evaluate(job_post, cover_letter) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)},
        {"role": "user", "content": "Reply ONLY in valid JSON: {\"is_acceptable\": true/false, \"feedback\": \"...\"}"}
    ]
    response = openai.chat.completions.create(model=evaluator_model, messages=messages)
    content = response.choices[0].message.content
    data = json.loads(content)
    return Evaluation(**data)

In [125]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [126]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [127]:
def requestLetter(job_posting, history):
    global updated_system_prompt
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 10:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"));
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            display(Markdown(f"## Updated system prompt:\n{updated_system_prompt}"))
            updated_system_prompt = system_prompt;
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [130]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting


Traceback (most recent call last):
  File "c:\Users\rutko\projects\agents\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rutko\projects\agents\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rutko\projects\agents\.venv\Lib\site-packages\gradio\blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rutko\projects\agents\.venv\Lib\site-packages\gradio\blocks.py", line 1700, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rutko\projects\agents\.venv\Lib\site-packages\gradio\utils.py", line 861, in async_wrapper
    response = await f(*args, **kwargs)
    